### Pre Process Steps to combine pax and flight data set

In [1]:
### Importing required python package
import pandas as pd
import numpy as np
import json          
from pyspark.sql import SparkSession 

### This code is for POC purpose. Dont deploy this in production environment without tuning further and 
### exception handling

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
261,application_1526256902441_5890,pyspark3,idle,Link,Link,✔


SparkSession available as 'spark'.


In [2]:
import time
starttime = time.time()

from time import gmtime, strftime
strftime("%Y-%m-%d %H:%M:%S", gmtime())

'2018-06-11 16:36:55'

In [3]:
## Define Increment Load or Full Load
loadtype = 'fullload'
rundate =  '05/20/2018'

import datetime

# naive datetime
rundate = datetime.datetime.strptime(rundate, '%m/%d/%Y')

print("The entire load will happen upto {}".format(rundate))
print("and this is {}".format(loadtype))


The entire load will happen upto 2018-05-20 00:00:00
and this is fullload

In [4]:
### Create spark session 
### This steps required to run from jupyter
spark = SparkSession.builder.appName("First SparkSession").getOrCreate()

In [5]:
### creating dataset to bring mod data.
### Table Name: meals_mod_data_mvp3_filtered
### Data is limited to 100 for testing. Change the query for bringing entire dataset

spark_mod = spark.sql("SELECT distinct q1.doctype, q1.fltKey, q1.channels FROM meals_mod_data_mvp3_filtered jv LATERAL VIEW JSON_TUPLE(jv.value, 'DocType','FltKey','channels') q1 AS doctype, fltKey, channels where q1.doctype = 'Order' LIMIT 100")

### Cleaning spark_mod data
spark_mod = spark_mod.withColumn('flight_number' , spark_mod.channels.substr(3, 4) ).withColumn('board_point' , spark_mod.channels.substr(8, 3) ).withColumn('operation_date_time' , spark_mod.channels.substr(12, 12) )

spark_mod = spark_mod.drop("channels" )

In [6]:
### testing spark_mod data 
#print(spark_mod.count())
spark_mod.show()

+-------+---------+-------------+-----------+-------------------+
|doctype|   fltKey|flight_number|board_point|operation_date_time|
+-------+---------+-------------+-----------+-------------------+
|  Order|  1855308|         0201|        DXB|       201801070830|
|  Order|  1861212|         0201|        DXB|       201801180830|
|  Order|  1886707|         0201|        DXB|       201803070830|
|  Order|  1853084|         0202|        JFK|       201801022300|
|  Order|999999992|         0202|        JFK|       201804072300|
|  Order|  1903260|         0202|        JFK|       201804062300|
|  Order|  1873066|         0074|        CDG|       201802091425|
|  Order|  1911053|         0074|        CDG|       201804211535|
|  Order|  1814953|         0081|        DXB|       201710231435|
|  Order|  1822261|         0081|        DXB|       201711061450|
|  Order|  1884668|         0081|        DXB|       201803031450|
|  Order|  1912419|         0087|        DXB|       201804240840|
|  Order| 

In [6]:
### Bringing the dataset for flights
spark_flight = spark.sql("select id, airline_designator, flight_number, CONCAT(SUBSTR(operation_date,1,4) , SUBSTR(operation_date,6,2) , SUBSTR(operation_date,9,2) , operation_time ) as flight_boarding_time, board_point, aircraft_type, class_configuration, pantry_code, crew_code, gate_number, baggage_concept, flight_status from macs_flight_mvp3 where operation_date >= '2018-03-20 00:00:00.0' union select id, airline_designator, flight_number, CONCAT(SUBSTR(operation_date,1,4) , SUBSTR(operation_date,6,2) , SUBSTR(operation_date,9,2) , operation_time ) as flight_boarding_time, board_point, aircraft_type, class_configuration, pantry_code, crew_code, gate_number, baggage_concept, flight_status from h_macs_flight_full_new where operation_date >= '2017-01-01 00:00:00.0' ")

In [15]:
spark_flight.registerTempTable("visitors")
spark_flight2 = spark.sql("select operation_date from visitors order by operation_date desc limit 10")

In [16]:
spark_flight2.show()

+-------------------+
|     operation_date|
+-------------------+
|     OPERATION_DATE|
|2018-08-30 00:00:00|
|2018-08-29 00:00:00|
|2018-08-28 00:00:00|
|2018-08-27 00:00:00|
|2018-08-26 00:00:00|
|2018-08-25 00:00:00|
|2018-08-24 00:00:00|
|2018-08-23 00:00:00|
|2018-08-22 00:00:00|
+-------------------+

In [6]:
### testing spark_flight data 
print(spark_flight.count())
#spark_flight.show()

1879277

In [7]:
### Bringing the dataset for pax
spark_pax = spark.sql("select pax_id, pax_booking_status, destination, cabin_class, transit_indicator, booked_class, travelled_class, seat_number as SeatNumber, pax_type, pax_group_code, boarding_date, upgrade_indicator, ticket_class, handicapped, unaccompanied_minor, priority_pax, check_in_city, date_of_birth, nationality, gender, country_of_birth, country_of_residence, mfl_id as id from h_macs_pax_joined_all_new where where boarding_date >= '2017-01-01 00:00:00.0' and upper(travelled_class) = 'J' UNION select pax_id, pax_booking_status, destination, cabin_class, transit_indicator, booked_class, travelled_class, seat_number as SeatNumber, pax_type, pax_group_code, boarding_date, upgrade_indicator, ticket_class, handicapped, unaccompanied_minor, priority_pax, check_in_city, date_of_birth, nationality, gender, country_of_birth, country_of_residence, mfl_id as id from macs_pax_joined_mvp3")

In [14]:
### testing spark_pax data 
print(spark_pax.count())
#spark_pax.show()

An error was encountered:
Session 250 did not reach idle status in time. Current status is busy.


In [8]:
spark_pax_flight_key  = [ "id" ]
spark_pax_flight = spark_pax.join(spark_flight, spark_pax_flight_key , "inner")

In [ ]:
### testing spark_pax_flight data 
#print(spark_pax_flight.count())
spark_pax_flight.show()

In [9]:
### Loading the pax and flights data into Hive table
spark_pax_flight.createOrReplaceTempView("spark_pax_flight_temp")
#sqlContext.sql("create table if not exists spark_pax_flight_mvp3 as select * from spark_pax_flight")
sqlContext.sql("insert overwrite table spark_pax_flight_mvp3 select * from spark_pax_flight_temp")

DataFrame[]

In [10]:
from time import gmtime, strftime
strftime("%Y-%m-%d %H:%M:%S", gmtime())

'2018-06-11 17:58:50'

In [ ]:
endtime = time.time()
diff = int ( endtime - starttime ) 
minutes, seconds = diff // 60, diff % 60

print("time taken: {} mins & {} secs ".format(minutes,seconds))